<a href="https://colab.research.google.com/github/hmsch/porcelan/blob/main/data/worm_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download data

In [ ]:
!wget 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE126954&format=file&file=GSE126954%5Fcell%5Fannotation%2Ecsv%2Egz' -O porcelan/data/GSE126954_cell_annotation.csv.gz
!gunzip porcelan/data/GSE126954_cell_annotation.csv.gz

!wget 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE126954&format=file&file=GSE126954%5Fgene%5Fannotation%2Ecsv%2Egz' -O porcelan/data/GSE126954_gene_annotation.csv.gz
!gunzip porcelan/data/GSE126954_gene_annotation.csv.gz

!wget 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE126954&format=file&file=GSE126954%5Fgene%5Fby%5Fcell%5Fcount%5Fmatrix%2Etxt%2Egz' -O porcelan/data/GSE126954_gene_by_cell_count_matrix.txt.gz
!gunzip porcelan/data/GSE126954_gene_by_cell_count_matrix.txt.gz

# Imports

In [1]:
import pandas as pd
import numpy as np
import pickle as pk
from ete3 import Tree
import dendropy
from scipy.io import mmread
import matplotlib.pyplot as plt

%cd porcelan/
data_path = './data'

/home/hschlueter/porcelan


# Create aggregate worm

Create aggregate worm by grouping cells from multiple individuals by lineage annotation and averaging gene expression profiles.

Only keep leaf cells.

Note: ordering of leaves matters -- here we sort alphabetically.

In [2]:
# full lineage tree
etree = Tree(f'{data_path}/celegans_1.nwk', format=1)
node_labels = np.array([n.name for n in etree.iter_search_nodes()])
print(etree.get_ascii())


                                                             /-ABALAAAALAL
                                                   /ABALAAAALA
                                                  |          \-ABALAAAALAR
                                          /ABALAAAAL
                                         |        |          /-ABALAAAALPA
                                         |         \ABALAAAALP
                                         |                   \-ABALAAAALPP
                                  /ABALAAAA
                                 |       |                   /-ABALAAAARLA
                                 |       |         /ABALAAAARL
                                 |       |        |          \-ABALAAAARLP
                                 |        \ABALAAAAR
                                 |                |          /-ABALAAAARRA
                                 |                 \ABALAAAARR
                                 |                           \-ABALAAAAR

In [3]:
cell_meta = pd.read_csv('/home/hschlueter/data/GSE126954/GSE126954_cell_annotation.csv')
cell_meta

,Unnamed: 0,cell,n.umi,time.point,batch,Size_Factor,cell.type,cell.subtype,plot.cell.type,raw.embryo.time,embryo.time,embryo.time.bin,raw.embryo.time.bin,lineage,passed_initial_QC_or_later_whitelisted
0,AAACCTGAGACAATAC-300.1.1,AAACCTGAGACAATAC-300.1.1,1630,300_minutes,Waterston_300_minutes,1.023195,Body_wall_muscle,BWM_head_row_1,BWM_head_row_1,360,380.0,330-390,330-390,MSxpappp,True
1,AAACCTGAGGGCTCTC-300.1.1,AAACCTGAGGGCTCTC-300.1.1,2319,300_minutes,Waterston_300_minutes,1.458210,NaN,NaN,NaN,260,220.0,210-270,210-270,MSxapaap,True
2,AAACCTGAGTGCGTGA-300.1.1,AAACCTGAGTGCGTGA-300.1.1,3719,300_minutes,Waterston_300_minutes,2.338283,NaN,NaN,NaN,270,230.0,210-270,270-330,NaN,True
3,AAACCTGAGTTGAGTA-300.1.1,AAACCTGAGTTGAGTA-300.1.1,4251,300_minutes,Waterston_300_minutes,2.659051,Body_wall_muscle,BWM_anterior,BWM_anterior,260,280.0,270-330,210-270,Dxap,True
4,AAACCTGCAAGACGTG-300.1.1,AAACCTGCAAGACGTG-300.1.1,1003,300_minutes,Waterston_300_minutes,0.629610,Ciliated_amphid_neuron,AFD,AFD,350,350.0,330-390,330-390,ABalpppapav/ABpraaaapav,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89696,TCTGAGACATGTCGAT-b02,TCTGAGACATGTCGAT-b02,585,mixed,Murray_b02,0.364709,Rectal_gland,Rectal_gland,Rectal_gland,390,700.0,> 650,390-450,NaN,True
89697,TCTGAGACATGTCTCC-b02,TCTGAGACATGTCTCC-b02,510,mixed,Murray_b02,0.323907,NaN,NaN,NaN,510,470.0,450-510,510-580,NaN,True
89698,TGGCCAGCACGAAGCA-b02,TGGCCAGCACGAAGCA-b02,843,mixed,Murray_b02,0.529174,NaN,NaN,NaN,400,470.0,450-510,390-450,NaN,True
89699,TGGCGCACAGGCAGTA-b02,TGGCGCACAGGCAGTA-b02,636,mixed,Murray_b02,0.397979,NaN,NaN,NaN,330,350.0,330-390,330-390,NaN,True


In [4]:
print(len(cell_meta))
pruned_meta = cell_meta[~cell_meta['lineage'].isna()]
print(len(pruned_meta))
pruned_meta = pruned_meta[pruned_meta['passed_initial_QC_or_later_whitelisted']]
print(len(pruned_meta))
pruned_meta

89701
47798
47798


,Unnamed: 0,cell,n.umi,time.point,batch,Size_Factor,cell.type,cell.subtype,plot.cell.type,raw.embryo.time,embryo.time,embryo.time.bin,raw.embryo.time.bin,lineage,passed_initial_QC_or_later_whitelisted
0,AAACCTGAGACAATAC-300.1.1,AAACCTGAGACAATAC-300.1.1,1630,300_minutes,Waterston_300_minutes,1.023195,Body_wall_muscle,BWM_head_row_1,BWM_head_row_1,360,380.0,330-390,330-390,MSxpappp,True
1,AAACCTGAGGGCTCTC-300.1.1,AAACCTGAGGGCTCTC-300.1.1,2319,300_minutes,Waterston_300_minutes,1.458210,NaN,NaN,NaN,260,220.0,210-270,210-270,MSxapaap,True
3,AAACCTGAGTTGAGTA-300.1.1,AAACCTGAGTTGAGTA-300.1.1,4251,300_minutes,Waterston_300_minutes,2.659051,Body_wall_muscle,BWM_anterior,BWM_anterior,260,280.0,270-330,210-270,Dxap,True
4,AAACCTGCAAGACGTG-300.1.1,AAACCTGCAAGACGTG-300.1.1,1003,300_minutes,Waterston_300_minutes,0.629610,Ciliated_amphid_neuron,AFD,AFD,350,350.0,330-390,330-390,ABalpppapav/ABpraaaapav,True
5,AAACCTGCAAGGTTCT-300.1.1,AAACCTGCAAGGTTCT-300.1.1,1319,300_minutes,Waterston_300_minutes,0.835505,Pharyngeal_neuron,I2_grandparent,I2_grandparent,260,270.0,270-330,210-270,ABalpappaa/ABarapapaa,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89659,AGGTCATCAGCCAGAA-b02,AGGTCATCAGCCAGAA-b02,526,mixed,Murray_b02,0.330184,Rectal_gland,rect_D,rect_D,380,355.0,330-390,330-390,ABplpappppp,True
89681,GACTGCGCACTTAAGC-b02,GACTGCGCACTTAAGC-b02,761,mixed,Murray_b02,0.475189,Rectal_gland,Rectal_gland,Rectal_gland,410,450.0,450-510,390-450,ABplpappppp/ABpxppppaap,True
89682,GCAAACTCACCTATCC-b02,GCAAACTCACCTATCC-b02,642,mixed,Murray_b02,0.403001,Rectal_gland,Rectal_gland,Rectal_gland,350,405.0,390-450,330-390,ABplpappppp/ABpxppppaap,True
89690,GTAGGCCTCTACTATC-b02,GTAGGCCTCTACTATC-b02,568,mixed,Murray_b02,0.356549,Rectal_cell,F_U,F_U,320,320.0,270-330,270-330,ABplppppapx,True


### Expand and deduplicate lineages

In [5]:
meta_lineages = pruned_meta['lineage'].str.split('/', expand=True)
meta_lineages.index = pruned_meta.index
meta_lineages

,0,1,2,3,4,5
0,MSxpappp,None,None,None,None,None
1,MSxapaap,None,None,None,None,None
3,Dxap,None,None,None,None,None
4,ABalpppapav,ABpraaaapav,None,None,None,None
5,ABalpappaa,ABarapapaa,None,None,None,None
...,...,...,...,...,...,...
89659,ABplpappppp,None,None,None,None,None
89681,ABplpappppp,ABpxppppaap,None,None,None,None
89682,ABplpappppp,ABpxppppaap,None,None,None,None
89690,ABplppppapx,None,None,None,None,None


In [6]:
lineage_patterns = meta_lineages.values.flatten()
lineage_patterns = lineage_patterns[~(lineage_patterns == None)]
lineage_patterns = np.unique(lineage_patterns)
print(len(lineage_patterns))
lineage_patterns

665


array(['28_cell_or_earlier', 'ABalaaaa', 'ABalaaaal', 'ABalaaaala',
       'ABalaaaalal', 'ABalaaaalp', 'ABalaaaar', 'ABalaaaarx',
       'ABalaaaarxp', 'ABalaaap', 'ABalaaapal', 'ABalaaapall',
       'ABalaaapalr', 'ABalaaapar', 'ABalaaappl', 'ABalaaappr',
       'ABalaaapprl', 'ABalaaapprr', 'ABalaaapx', 'ABalaapa', 'ABalaapaa',
       'ABalaapaaa', 'ABalaapaaar', 'ABalaapaap', 'ABalaapap',
       'ABalaapapp', 'ABalaapappp', 'ABalaapp', 'ABalaappa', 'ABalaappap',
       'ABalaappapa', 'ABalaappp', 'ABalaapppa', 'ABalaapppaa',
       'ABalaapppp', 'ABalaappppa', 'ABalaappppaa', 'ABalaappppp',
       'ABalaax', 'ABalapaa', 'ABalapaaa', 'ABalapaaaa', 'ABalapaaaaa',
       'ABalapaaap', 'ABalapaaapa', 'ABalapaap', 'ABalapaapa',
       'ABalapaapaa', 'ABalapaapp', 'ABalapaappa', 'ABalapaappaa',
       'ABalapaappp', 'ABalapapa', 'ABalappa', 'ABalappaa', 'ABalappaap',
       'ABalappaapa', 'ABalappap', 'ABalappapa', 'ABalappapaa',
       'ABalappapp', 'ABalappappa', 'ABalapppa', 'ABalappp

In [7]:
from tqdm import tqdm

lineages = pd.Series(lineage_patterns)
regex_lineages = lineages.str.replace('x', '[xaplrvd]')
matches = pd.DataFrame(np.zeros((len(lineages), len(lineages))).astype(bool))
matches.columns = regex_lineages
for rexp in tqdm(regex_lineages):
  matches[rexp] = lineages.str.fullmatch(rexp)
matches.index = lineages
matches

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 665/665 [00:00<00:00, 2779.95it/s]


,28_cell_or_earlier,ABalaaaa,ABalaaaal,ABalaaaala,ABalaaaalal,ABalaaaalp,ABalaaaar,ABalaaaar[xaplrvd],ABalaaaar[xaplrvd]p,ABalaaap,...,MS[xaplrvd]pppap,MS[xaplrvd]pppa[xaplrvd],MS[xaplrvd]pppp,MS[xaplrvd]ppppa,MS[xaplrvd]ppppp,MS[xaplrvd]pppp[xaplrvd],Z2,Z3:pseudotime_bin_1,Z3:pseudotime_bin_2,Z3:pseudotime_bin_3
28_cell_or_earlier,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABalaaaa,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABalaaaal,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABalaaaala,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABalaaaalal,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSxppppx,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
Z2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
Z3:pseudotime_bin_1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
Z3:pseudotime_bin_2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [8]:
# nothing unmatched
(matches.sum(axis=1) == 0).sum()

0

In [9]:
# for some reason there are versions of lineages subsuming other lineages
print((matches.sum(axis=1) != 1).sum())

sub = matches[matches.sum(axis=1) != 1]
for idx, row in sub.iterrows():
    print(idx, 'matches', sub.columns.values[row.values])

18
ABalapaa matches ['ABalapaa' 'ABalap[xaplrvd]a']
ABalapapa matches ['ABalapapa' 'ABalap[xaplrvd]pa']
ABalappa matches ['ABalappa' 'ABalap[xaplrvd]a']
ABalapppa matches ['ABalapppa' 'ABalap[xaplrvd]pa']
ABarpaapaa matches ['ABarpaapaa' 'ABarpaap[xaplrvd][xaplrvd]']
ABarpaapap matches ['ABarpaapap' 'ABarpaap[xaplrvd][xaplrvd]']
ABarpaappx matches ['ABarpaapp[xaplrvd]' 'ABarpaap[xaplrvd][xaplrvd]']
Dxaap matches ['D[xaplrvd]aap' 'D[xaplrvd][xaplrvd]a[xaplrvd]']
Dxapa matches ['D[xaplrvd]apa' 'D[xaplrvd]ap[xaplrvd]']
Dxapp matches ['D[xaplrvd]app' 'D[xaplrvd]ap[xaplrvd]']
Dxpap matches ['D[xaplrvd]pap' 'D[xaplrvd][xaplrvd]a[xaplrvd]']
Dxxaa matches ['D[xaplrvd][xaplrvd]aa' 'D[xaplrvd][xaplrvd]a[xaplrvd]']
MSappaaa matches ['MSappaaa' 'MS[xaplrvd]ppaaa']
MSpppaaa matches ['MSpppaaa' 'MS[xaplrvd]ppaaa']
MSxpppaa matches ['MS[xaplrvd]pppaa' 'MS[xaplrvd]pppa[xaplrvd]']
MSxpppap matches ['MS[xaplrvd]pppap' 'MS[xaplrvd]pppa[xaplrvd]']
MSxppppa matches ['MS[xaplrvd]ppppa' 'MS[xaplrvd]pppp[xapl

In [10]:
sorter = np.argsort(- matches.sum(axis=0))
ordered_cols = matches.columns[sorter]
ordered_matches = matches[ordered_cols].iloc[sorter]
match_counts = ordered_matches.sum(axis=0)
match_counts

D[xaplrvd][xaplrvd]a[xaplrvd]    4
ABarpaap[xaplrvd][xaplrvd]       4
ABalap[xaplrvd]a                 3
MS[xaplrvd]pppp[xaplrvd]         3
ABalap[xaplrvd]pa                3
                                ..
ABarappaapa                      1
ABarappap                        1
ABarappapa                       1
ABarappapapp                     1
Z3:pseudotime_bin_3              1
Length: 665, dtype: int64

In [11]:
multi_match = match_counts[match_counts > 1]
print(len(multi_match))
multi_match

8


D[xaplrvd][xaplrvd]a[xaplrvd]    4
ABarpaap[xaplrvd][xaplrvd]       4
ABalap[xaplrvd]a                 3
MS[xaplrvd]pppp[xaplrvd]         3
ABalap[xaplrvd]pa                3
D[xaplrvd]ap[xaplrvd]            3
MS[xaplrvd]pppa[xaplrvd]         3
MS[xaplrvd]ppaaa                 3
dtype: int64

In [12]:
# multi-matches are not nested, so no need for recursive replacement
ordered_matches.iloc[:8, :8].sum(axis=0)

D[xaplrvd][xaplrvd]a[xaplrvd]    1
ABarpaap[xaplrvd][xaplrvd]       1
ABalap[xaplrvd]a                 1
MS[xaplrvd]pppp[xaplrvd]         1
ABalap[xaplrvd]pa                1
D[xaplrvd]ap[xaplrvd]            1
MS[xaplrvd]pppa[xaplrvd]         1
MS[xaplrvd]ppaaa                 1
dtype: int64

In [13]:
# find rows matching just one regex (themself)
good_lineages = np.argwhere((matches.sum(axis=1) == 1).values).flatten()
# remove columns (regex) for bad lineages
unique_matches = matches.iloc[:, good_lineages]
assert (unique_matches.sum(axis=1) == 1).all()

metalineage_to_regex = {}
for lineage in unique_matches.index:
  metalineage_to_regex[lineage] = unique_matches.columns[unique_matches.loc[lineage].argmax()]

meta_regex_lineages = meta_lineages.replace(metalineage_to_regex)
meta_regex_lineages

,0,1,2,3,4,5
0,MS[xaplrvd]pappp,None,None,None,None,None
1,MS[xaplrvd]apaap,None,None,None,None,None
3,D[xaplrvd]ap,None,None,None,None,None
4,ABalpppapav,ABpraaaapav,None,None,None,None
5,ABalpappaa,ABarapapaa,None,None,None,None
...,...,...,...,...,...,...
89659,ABplpappppp,None,None,None,None,None
89681,ABplpappppp,ABp[xaplrvd]ppppaap,None,None,None,None
89682,ABplpappppp,ABp[xaplrvd]ppppaap,None,None,None,None
89690,ABplppppap[xaplrvd],None,None,None,None,None


In [14]:
meta_regex_lineages_joined = meta_regex_lineages.apply(
    lambda x: '|'.join(x.dropna().astype(str)),
    axis=1
)
meta_regex_lineages_joined

0                       MS[xaplrvd]pappp
1                       MS[xaplrvd]apaap
3                           D[xaplrvd]ap
4                ABalpppapav|ABpraaaapav
5                  ABalpappaa|ABarapapaa
                      ...               
89659                        ABplpappppp
89681    ABplpappppp|ABp[xaplrvd]ppppaap
89682    ABplpappppp|ABp[xaplrvd]ppppaap
89690                ABplppppap[xaplrvd]
89692                        ABprpppaaaa
Length: 47798, dtype: object

In [15]:
meta_regex_lineages_joined_set = meta_regex_lineages_joined.unique()
meta_regex_lineages_joined_set_to_cell_ids = {}
for reg in meta_regex_lineages_joined_set:
  meta_regex_lineages_joined_set_to_cell_ids[reg] = meta_regex_lineages_joined[meta_regex_lineages_joined == reg].index.values

print(len(meta_regex_lineages_joined_set))

499


### Extract leaves for lineages

In [16]:
pruned_tree = dendropy.Tree.get(path=f'{data_path}/celegans_1.nwk', schema='newick')
# add taxa to internal nodes
for node in pruned_tree.internal_nodes():
  node.taxon = dendropy.Taxon(node.label)
leaf_labels = np.array([t.taxon.label for t in pruned_tree.leaf_nodes()])
leaf_labels = pd.Series(leaf_labels).sort_values()

In [17]:
leaf_matches = pd.DataFrame(np.zeros((len(leaf_labels), len(meta_regex_lineages_joined_set))).astype(bool))
leaf_matches.columns = meta_regex_lineages_joined_set
leaf_matches.index = leaf_labels
for rexp in tqdm(meta_regex_lineages_joined_set):
  leaf_matches[rexp] = leaf_labels.str.fullmatch(str.upper(rexp)).values
leaf_matches

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 499/499 [00:00<00:00, 2798.85it/s]


,MS[xaplrvd]pappp,MS[xaplrvd]apaap,D[xaplrvd]ap,ABalpppapav|ABpraaaapav,ABalpappaa|ABarapapaa,ABp[xaplrvd]aapppa,ABalap[xaplrvd]papa|ABp[xaplrvd]paaappa,D[xaplrvd][xaplrvd]a[xaplrvd],ABalppppa|ABpraaapa,MS[xaplrvd]pppp[xaplrvd],...,C[xaplrvd]a,ABalaa[xaplrvd]|ABalap[xaplrvd]a,MS[xaplrvd]a,ABarpp[xaplrvd]a,MS[xaplrvd],ABalaa[xaplrvd],ABp[xaplrvd]app,C[xaplrvd]ap,ABp[xaplrvd]apa,ABp[xaplrvd]pa
ABALAAAALAL,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAALAR,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAALPA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAALPP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAARLA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSPPPPAP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPPPA,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
MSPPPPPP,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
P4A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
meta_regex_lineages_joined_set_to_leaf = {}
matched_leaves = set()
for reg in meta_regex_lineages_joined_set:
  if leaf_matches[reg].any():
    leaf = leaf_matches.index[np.argmax(leaf_matches[reg])]
    meta_regex_lineages_joined_set_to_leaf[reg] = leaf
    matched_leaves.add(leaf)

# drop matched columns
leaf_matches = leaf_matches[leaf_matches.columns[~leaf_matches.columns.isin(meta_regex_lineages_joined_set_to_leaf.keys())]]
# drop matched rows
leaf_matches = leaf_matches.iloc[~leaf_matches.index.isin(matched_leaves)]
leaf_matches

,MS[xaplrvd]apaap,D[xaplrvd]ap,ABalpappaa|ABarapapaa,ABalap[xaplrvd]papa|ABp[xaplrvd]paaappa,ABalppppa|ABpraaapa,ABaraaaapa,ABp[xaplrvd]ppapaa,ABalpaapa|ABaraaapa,ABalaaaala|ABalaapaaa,ABp[xaplrvd]paaaaa,...,C[xaplrvd]a,ABalaa[xaplrvd]|ABalap[xaplrvd]a,MS[xaplrvd]a,ABarpp[xaplrvd]a,MS[xaplrvd],ABalaa[xaplrvd],ABp[xaplrvd]app,C[xaplrvd]ap,ABp[xaplrvd]apa,ABp[xaplrvd]pa
ABALAAAALAR,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAALPA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAALPP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAARLA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAARRA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSPPPPAP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPPPA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPPPP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
P4A,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
leaf_matches.any().any()  # no matches left

False

In [20]:
to_prune =  leaf_matches.index.values
pruned_tree.prune_taxa_with_labels(to_prune)
pruned_tree.update_taxon_namespace()
leaf_labels = np.array([t.taxon.label for t in pruned_tree.leaf_nodes()])
print('to_prune', len(to_prune), 'remaining', len(leaf_labels))

to_prune 520 remaining 335


In [21]:
unmatched_meta_regex = leaf_matches.columns
leaf_labels = pd.Series(leaf_labels).sort_values()
leaf_matches = pd.DataFrame(np.zeros((len(leaf_labels), len(unmatched_meta_regex))).astype(bool))
leaf_matches.columns = unmatched_meta_regex
leaf_matches.index = leaf_labels
for rexp in tqdm(unmatched_meta_regex):
  leaf_matches[rexp] = leaf_labels.str.fullmatch(str.upper(rexp)).values
leaf_matches

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 349/349 [00:00<00:00, 4585.98it/s]


,MS[xaplrvd]apaap,D[xaplrvd]ap,ABalpappaa|ABarapapaa,ABalap[xaplrvd]papa|ABp[xaplrvd]paaappa,ABalppppa|ABpraaapa,ABaraaaapa,ABp[xaplrvd]ppapaa,ABalpaapa|ABaraaapa,ABalaaaala|ABalaapaaa,ABp[xaplrvd]paaaaa,...,C[xaplrvd]a,ABalaa[xaplrvd]|ABalap[xaplrvd]a,MS[xaplrvd]a,ABarpp[xaplrvd]a,MS[xaplrvd],ABalaa[xaplrvd],ABp[xaplrvd]app,C[xaplrvd]ap,ABp[xaplrvd]apa,ABp[xaplrvd]pa
ABALAAAALAL,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAALP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAARLP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAARR,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAPALL,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSPPPAA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPAP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPPA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPPP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
for reg in leaf_matches.columns:
  if leaf_matches[reg].any():
    leaf = leaf_matches.index[np.argmax(leaf_matches[reg])]
    meta_regex_lineages_joined_set_to_leaf[reg] = leaf
    matched_leaves.add(leaf)

# drop matched columns
leaf_matches = leaf_matches[leaf_matches.columns[~leaf_matches.columns.isin(meta_regex_lineages_joined_set_to_leaf.keys())]]
# drop matched rows
leaf_matches = leaf_matches.iloc[~leaf_matches.index.isin(matched_leaves)]
leaf_matches

,MS[xaplrvd]apaap,ABalpappaa|ABarapapaa,ABalppppa|ABpraaapa,ABp[xaplrvd]ppapaa,ABalpaapa|ABaraaapa,ABp[xaplrvd]ppppap,ABprpappap,ABalaapppp|ABalapaapp,C[xaplrvd]ppa,ABp[xaplrvd]ppppa,...,C[xaplrvd]a,ABalaa[xaplrvd]|ABalap[xaplrvd]a,MS[xaplrvd]a,ABarpp[xaplrvd]a,MS[xaplrvd],ABalaa[xaplrvd],ABp[xaplrvd]app,C[xaplrvd]ap,ABp[xaplrvd]apa,ABp[xaplrvd]pa
ABALAAAPPL,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAPAAP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAPAPPAA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAPPPPAA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALPAPPAAP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALPPAPPPA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABARAAAAAP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABARAAPPPAA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABARAAPPPP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABARAPAAAA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
leaf_matches.any().any()  # no matches left

False

In [24]:
to_prune =  leaf_matches.index.values
pruned_tree.prune_taxa_with_labels(to_prune)
pruned_tree.update_taxon_namespace()
leaf_labels = np.array([t.taxon.label for t in pruned_tree.leaf_nodes()])
print('to_prune', len(to_prune), 'remaining', len(leaf_labels))

to_prune 53 remaining 295


In [25]:
unmatched_meta_regex = leaf_matches.columns
leaf_labels = pd.Series(leaf_labels).sort_values()
leaf_matches = pd.DataFrame(np.zeros((len(leaf_labels), len(unmatched_meta_regex))).astype(bool))
leaf_matches.columns = unmatched_meta_regex
leaf_matches.index = leaf_labels
for rexp in tqdm(unmatched_meta_regex):
  leaf_matches[rexp] = leaf_labels.str.fullmatch(str.upper(rexp)).values
leaf_matches

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 215/215 [00:00<00:00, 4835.44it/s]


,MS[xaplrvd]apaap,ABalpappaa|ABarapapaa,ABalppppa|ABpraaapa,ABp[xaplrvd]ppapaa,ABalpaapa|ABaraaapa,ABp[xaplrvd]ppppap,ABprpappap,ABalaapppp|ABalapaapp,C[xaplrvd]ppa,ABp[xaplrvd]ppppa,...,C[xaplrvd]a,ABalaa[xaplrvd]|ABalap[xaplrvd]a,MS[xaplrvd]a,ABarpp[xaplrvd]a,MS[xaplrvd],ABalaa[xaplrvd],ABp[xaplrvd]app,C[xaplrvd]ap,ABp[xaplrvd]apa,ABp[xaplrvd]pa
ABALAAAALAL,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAALP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAARLP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAARR,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
ABALAAAPALL,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSPPAPP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPAA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPAP,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
MSPPPPA,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
for reg in leaf_matches.columns:
  if leaf_matches[reg].any():
    leaf = leaf_matches.index[np.argmax(leaf_matches[reg])]
    meta_regex_lineages_joined_set_to_leaf[reg] = leaf
    matched_leaves.add(leaf)

# drop matched columns
leaf_matches = leaf_matches[leaf_matches.columns[~leaf_matches.columns.isin(meta_regex_lineages_joined_set_to_leaf.keys())]]
# drop matched rows
leaf_matches = leaf_matches.iloc[~leaf_matches.index.isin(matched_leaves)]
leaf_matches  # nothing left

,MS[xaplrvd]apaap,ABalpappaa|ABarapapaa,ABalppppa|ABpraaapa,ABp[xaplrvd]ppapaa,ABalpaapa|ABaraaapa,ABp[xaplrvd]ppppap,ABprpappap,ABalaapppp|ABalapaapp,C[xaplrvd]ppa,ABp[xaplrvd]ppppa,...,C[xaplrvd]a,ABalaa[xaplrvd]|ABalap[xaplrvd]a,MS[xaplrvd]a,ABarpp[xaplrvd]a,MS[xaplrvd],ABalaa[xaplrvd],ABp[xaplrvd]app,C[xaplrvd]ap,ABp[xaplrvd]apa,ABp[xaplrvd]pa


In [27]:
len(set(meta_regex_lineages_joined_set_to_leaf.values()))

295

In [28]:
meta_regex_lineages_joined

0                       MS[xaplrvd]pappp
1                       MS[xaplrvd]apaap
3                           D[xaplrvd]ap
4                ABalpppapav|ABpraaaapav
5                  ABalpappaa|ABarapapaa
                      ...               
89659                        ABplpappppp
89681    ABplpappppp|ABp[xaplrvd]ppppaap
89682    ABplpappppp|ABp[xaplrvd]ppppaap
89690                ABplppppap[xaplrvd]
89692                        ABprpppaaaa
Length: 47798, dtype: object

In [29]:
cell_to_leaf_df = pd.DataFrame(meta_regex_lineages_joined, columns=['regex_joined'])
cell_to_leaf_df.index.name = 'og_idx'
cell_to_leaf_df = cell_to_leaf_df.loc[cell_to_leaf_df['regex_joined'].isin(meta_regex_lineages_joined_set_to_leaf.keys())]
cell_to_leaf_df['leaf'] = cell_to_leaf_df['regex_joined'].replace(meta_regex_lineages_joined_set_to_leaf)
cell_to_leaf_df

,regex_joined,leaf
og_idx,,
0,MS[xaplrvd]pappp,MSAPAPPP
3,D[xaplrvd]ap,DPAP
4,ABalpppapav|ABpraaaapav,ABALPPPAPAV
6,ABp[xaplrvd]aapppa,ABPLAAPPPA
7,ABalap[xaplrvd]papa|ABp[xaplrvd]paaappa,ABALAPPPAPA
...,...,...
89659,ABplpappppp,ABPLPAPPPPP
89681,ABplpappppp|ABp[xaplrvd]ppppaap,ABPLPAPPPPP
89682,ABplpappppp|ABp[xaplrvd]ppppaap,ABPLPAPPPPP


In [30]:
pruned_tree.write(path=f'{data_path}/preprocessed/worm_avg_leaves_pruned.nwk', schema='newick')

# Gene expression pre-processing

In [31]:
# R sparse matrix
expression_adata = mmread(f'{data_path}/GSE126954_gene_by_cell_count_matrix.txt')
print(expression_adata.shape)

all_gene_names = pd.read_csv(f'{data_path}/GSE126954_gene_annotation.csv')
all_gene_names = all_gene_names.id.values
print(len(all_gene_names))

expression_df = pd.DataFrame(expression_adata.toarray()[:, cell_to_leaf_df.index].T,
                             index=cell_to_leaf_df.index, columns=all_gene_names)
expression_df['leaf'] = cell_to_leaf_df['leaf']
expression_df

(20222, 89701)
20222


,WBGene00010957,WBGene00010958,WBGene00010959,WBGene00010960,WBGene00010961,WBGene00000829,WBGene00010962,WBGene00010963,WBGene00010964,WBGene00010965,...,WBGene00001780,WBGene00001782,WBGene00001783,WBGene00021598,WBGene00021597,WBGene00021596,WBGene00021595,WBGene00021594,WBGene00007064,leaf
og_idx,,,,,,,,,,,,,,,,,,,,,
0,5,0,0,7,3,5,8,3,5,4,...,0,0,0,0,0,0,1,0,0,MSAPAPPP
3,26,1,14,52,5,17,38,6,17,17,...,0,0,0,0,0,0,0,0,0,DPAP
4,4,0,2,7,0,0,1,2,3,7,...,0,0,0,0,0,0,0,0,0,ABALPPPAPAV
6,73,0,38,76,9,17,49,11,31,39,...,0,0,0,0,0,0,0,0,0,ABPLAAPPPA
7,7,0,2,0,1,0,1,0,7,3,...,0,0,0,0,0,0,0,0,0,ABALAPPPAPA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89659,12,1,4,12,1,5,10,1,3,11,...,0,0,0,0,0,0,0,0,1,ABPLPAPPPPP
89681,8,0,0,2,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,ABPLPAPPPPP
89682,16,1,2,6,2,5,18,3,7,9,...,0,0,0,0,0,0,0,0,0,ABPLPAPPPPP


In [38]:
leaf_expression = expression_df.groupby(['leaf']).mean()
leaf_expression.to_csv(f'{data_path}/preprocessed/worm_avg_leaves_cells.txt', index=True, columns=[], header=False)
leaf_expression

,WBGene00010957,WBGene00010958,WBGene00010959,WBGene00010960,WBGene00010961,WBGene00000829,WBGene00010962,WBGene00010963,WBGene00010964,WBGene00010965,...,WBGene00013175,WBGene00001780,WBGene00001782,WBGene00001783,WBGene00021598,WBGene00021597,WBGene00021596,WBGene00021595,WBGene00021594,WBGene00007064
leaf,,,,,,,,,,,,,,,,,,,,,
ABALAAAALAL,4.441176,0.147059,1.205882,2.794118,0.411765,0.617647,3.617647,0.647059,2.147059,2.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.029412,0.029412,0.000000,0.058824
ABALAAAALP,7.612613,0.450450,2.738739,7.135135,0.882883,1.990991,7.882883,1.306306,3.738739,4.972973,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.036036,0.000000,0.018018
ABALAAAARLP,4.830579,0.289256,1.714876,2.896694,0.442149,1.342975,3.859504,0.772727,2.628099,3.826446,...,0.0,0.0,0.0,0.000000,0.0,0.004132,0.004132,0.024793,0.000000,0.016529
ABALAAAARR,8.717742,0.282258,2.790323,7.266129,0.766129,2.241935,9.185484,1.145161,3.814516,6.290323,...,0.0,0.0,0.0,0.000000,0.0,0.008065,0.000000,0.024194,0.000000,0.048387
ABALAAAPALL,4.458015,0.335878,1.419847,2.404580,0.427481,1.282443,3.328244,0.580153,2.137405,3.748092,...,0.0,0.0,0.0,0.000000,0.0,0.137405,0.000000,0.015267,0.007634,0.053435
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSPPAPP,19.070352,0.753769,6.884422,24.060302,1.839196,4.437186,20.507538,2.648241,7.914573,11.110553,...,0.0,0.0,0.0,0.000000,0.0,0.005025,0.015075,0.256281,0.000000,0.040201
MSPPPAA,15.343284,0.626866,5.676617,17.975124,1.437811,4.109453,17.791045,2.154229,7.079602,10.557214,...,0.0,0.0,0.0,0.014925,0.0,0.014925,0.000000,0.213930,0.000000,0.029851
MSPPPAP,12.065116,0.451163,4.706977,13.925581,1.241860,2.855814,13.320930,1.739535,5.390698,8.655814,...,0.0,0.0,0.0,0.000000,0.0,0.009302,0.000000,0.130233,0.000000,0.023256


In [33]:
# there are no duplicates!
unq, count = np.unique(leaf_expression.values, axis=0, return_counts=True)
len(unq[count>1])

0

In [34]:
# remove rare genes
print(leaf_expression.shape)
leaf_expression = leaf_expression.iloc[:, ((leaf_expression > 0).sum(axis=0) >= 10).values.flatten()]
print(leaf_expression.shape)
keep_gene_names = leaf_expression.columns
gene_counts = leaf_expression.values

# L1 normalization on cells to [0, 10000]
gene_counts = gene_counts / gene_counts.sum(axis=1).reshape(-1, 1)
gene_counts *= 10000
# log2(1 + x) tranform
gene_counts = np.log2(1 + gene_counts)

df = pd.DataFrame(gene_counts)
df.columns = keep_gene_names
df.to_csv(f'{data_path}/preprocessed/worm_avg_leaves_normalized_log_counts.txt', index=False)
df

(295, 20222)
(295, 13918)


,WBGene00010957,WBGene00010958,WBGene00010959,WBGene00010960,WBGene00010961,WBGene00000829,WBGene00010962,WBGene00010963,WBGene00010964,WBGene00010965,...,WBGene00001777,WBGene00001787,WBGene00001779,WBGene00001783,WBGene00021598,WBGene00021597,WBGene00021596,WBGene00021595,WBGene00021594,WBGene00007064
0,5.679704,1.413686,3.872454,5.027651,2.500720,2.998108,5.390207,3.056991,4.660888,4.562714,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.414497,0.414497,0.000000,0.736101
1,5.697354,2.004026,4.271124,5.605763,2.786995,3.838892,5.746732,3.282798,4.700039,5.097754,...,0.0,0.084368,0.0,0.000000,0.0,0.000000,0.000000,0.311380,0.000000,0.164075
2,5.596016,1.939782,4.155122,4.878004,2.415693,3.824702,5.279720,3.100548,4.742622,5.267631,...,0.0,0.000000,0.0,0.000000,0.0,0.057306,0.057306,0.313973,0.000000,0.216715
3,5.749838,1.438237,4.162172,5.492416,2.495987,3.866062,5.823873,2.988646,4.591440,5.289326,...,0.0,0.000000,0.0,0.000000,0.0,0.068814,0.000000,0.197315,0.000000,0.370861
4,5.630195,2.219177,4.040549,4.764241,2.499130,3.903070,5.218408,2.870934,4.600937,5.385456,...,0.0,0.000000,0.0,0.000000,0.0,1.319473,0.000000,0.221808,0.115163,0.661458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,6.484244,2.169628,5.042565,6.816221,3.253647,4.432817,6.587937,3.732639,5.238033,5.716342,...,0.0,0.000000,0.0,0.000000,0.0,0.033267,0.097586,1.130723,0.000000,0.246855
291,6.226306,2.006483,4.824248,6.451877,2.985914,4.377454,6.437197,3.507342,5.132758,5.695638,...,0.0,0.000000,0.0,0.100113,0.0,0.100113,0.000000,1.021449,0.000000,0.193728
292,6.195602,1.888819,4.868080,6.399865,3.077431,4.178853,6.336598,3.513884,5.057472,5.724234,...,0.0,0.000000,0.0,0.000000,0.0,0.078253,0.000000,0.832153,0.000000,0.188206
293,6.382175,2.052573,4.934020,6.803261,3.334613,4.438021,6.664282,3.740560,5.199144,5.647008,...,0.0,0.000000,0.0,0.000000,0.0,0.046083,0.000000,0.932690,0.000000,0.176110


# Cell types

In [35]:
cell_types = cell_to_leaf_df.copy()
cell_types['type'] = cell_meta.iloc[cell_to_leaf_df.index]['cell.type']
cell_types = cell_types.groupby('leaf')['type'].unique()
cell_types

leaf
ABALAAAALAL                                 [nan]
ABALAAAALP          [nan, ABarpaaa_lineage, Glia]
ABALAAAARLP                                 [nan]
ABALAAAARR     [nan, Pharyngeal_intestinal_valve]
ABALAAAPALL                                [Glia]
                              ...                
MSPPAPP                   [Body_wall_muscle, nan]
MSPPPAA                              [nan, Z1_Z4]
MSPPPAP              [nan, GLR, Body_wall_muscle]
MSPPPPA                        [Body_wall_muscle]
MSPPPPP                   [Body_wall_muscle, nan]
Name: type, Length: 295, dtype: object

In [36]:
def f(types):
  if len(types) == 0:
    return np.nan
  if types[0] is None or types[0] == np.nan:
    return f(types[1:])
  return types[0]

cell_types = pd.DataFrame(cell_types.map(f))
cell_types

,type
leaf,
ABALAAAALAL,NaN
ABALAAAALP,NaN
ABALAAAARLP,NaN
ABALAAAARR,NaN
ABALAAAPALL,Glia
...,...
MSPPAPP,Body_wall_muscle
MSPPPAA,NaN
MSPPPAP,NaN


In [37]:
cell_types.to_csv(f'{data_path}/preprocessed/worm_avg_leaves_cell_types.csv')